<a href="https://colab.research.google.com/github/project-ccap/project-ccap.github.io/blob/master/notebooks/2021Roelofs_ERP_bilingual_lemret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 他言語プライミング課題での事象関連電位 （ERP) のシミュレーション

<font size="+2" style="bold" color="teal">
Roelfs (2016) Electrophysiology of cross-language interference and facilitation in picture naming, Cortex 76, 1-16.
</font>
    
絵画命名課題 の 反応時間は，無関連妨害語と比較して， 妨害語が意味的に関連する場合長くなる。
しかし，妨害語が他言語の単語名であった場合，反応時間は短くなる。
これらの効果は、他言語妨害語は，それ自体が競合するのではなく，対象言語での対応語を活性化させることで、意味的な干渉効果と翻訳促進効果をもたらしていることを示唆している。
プライミング課題を用いた，意味的干渉効果と翻訳効果の根底にあるという予測を検証する事象関連脳電位（ERP）研究で，意味的干渉効果と翻訳促進効果が得られた。
さらに、絵画ー言葉刺激では N400 応答が得られた。
だが、その振幅は意味的試行と翻訳試行の方が無関連試行よりも小さく、意味的干渉と翻訳促進のプライミングがR T効果の根底にあることを示している。


- filename: `2016Roelofs_ERP_bilingual_lemret.ipynb`
- note: SIMULATION OF LEMMA RETRIEVAL IN SPEAKING BILINGUAL NAMING
- author: Shin Asakawa <asakawa@ieee.org>
- date: 2020-0103

---
## Original resorces: 
- date: 2016
- author(s):Roelofs, A., Piai, V., Garrido Rodriguez, G., & Chwilla, D. J. 
- title: Electrophysiology of cross-language interference and facilitation in picture naming.
- journal: Cortex
- volume: 76
- pages: 1-16
- Abstract:
Disagreement exists about how bilingual speakers select words, in particular, whether words in another language compete, or competition is restricted to a target language, or no competition occurs. 
Evidence that competition occurs but is restricted to a target language comes from response time (RT) effects obtained when speakers name pictures in one language while trying to ignore distractor words in another language. 
Compared to unrelated distractor words, RT is longer when the picture name and distractor are semantically related, but RT is shorter when the distractor is the translation of the name of the picture in the other language. 
These effects suggest that distractor words from another language do not compete themselves but activate their counterparts in the target language, thereby yielding the semantic interference and translation facilitation effects. 
Here, we report an event-related brain potential (ERP) study testing the prediction that priming underlies both of these effects. 
The RTs showed semantic interference and translation facilitation effects. 
Moreover, the picture-word stimuli yielded an N400 response, whose amplitude was smaller on semantic and translation trials than on unrelated trials, providing evidence that interference and facilitation priming underlie the RT effects. 
We present the results of computer simulations showing the utility of a within-language competition account of our findings.

In [1]:
import numpy as np
import sys
import os

# 表示精度桁数の設定
#np.set_printoptions(suppress=False, formatter={'float': '{:7.4f}'.format})
#np.set_printoptions(suppress=False, formatter={'float': '{:6.3f}'.format})
np.set_printoptions(suppress=False, formatter={'float': '{:5.2f}'.format})

import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import SVG, display


In [2]:
STEP_SIZE = 25
N_STEPs = int((1000+STEP_SIZE)/STEP_SIZE)  # /* trunc after 41 steps */
N_CONCEPTs = 6
N_LEMMAs = 12
N_DCONDs = 4

# parameter set
VERIFICATION_SPEED = 25          # ms per link
SEM_rate = 0.0101 * STEP_SIZE    # prop per step_size ms
LEM_rate = 0.0074 * STEP_SIZE    # prop per step_size ms
DECAY_rate = 0.0240 * STEP_SIZE  # prop per step_size ms
EXTIN = 0.1965 * STEP_SIZE       # act_units per step_size ms
DISTRACTOR_DURATION = 100        # ms
CRIT_DIFF = 1.6                  # act_units

Y = 1.0    # connection present
N = 0.0    # connection absent
Y1 = 1.0   # connection present, L1
Y2 = 0.5   # connection present, weaker L2
REL = 0    # semantically related condition
UNR = 1    # unrelated condition
TRA = 2    # translation condition
CTR = 3    # control condition

# labeling network nodes
# L2 English, the target language for picture naming
ANIMAL    = 0
DUCK      = 1
HORSE     = 2
FURNITURE = 3
TABLE     = 4
CHAIR     = 5
# L1 Dutch, the language of the distractor words
# Dutch words are translation equivalents of the English words
DIER   = 6   # animal
EEND   = 7   # duck
PAARD  = 8   # horse 
MEUBEL = 9   # furniture
TAFEL  = 10  # table
STOEL  = 11  # chair


def make_weight():
    
    # connections conceptual network, shared between languages
    SEM_con = np.array( #double SEM_con[N_CONCEPTs][N_CONCEPTs] =
        #ANIMAL DUCK HORSE FURN TABLE CHAIR
        [[N, Y, Y, N, N, N],  # ANIMAL
         [Y, N, Y, N, N, N],  # DUCK
         [Y, Y, N, N, N, N],  # HORSE
         [N, N, N, N, Y, Y],  # FURNITURE
         [N, N, N, Y, N, Y],  # TABLE
         [N, N, N, Y, Y, N]], # CHAIR
        dtype=np.float)
    
    # connections between concept and lemma nodes
    LEM_con = np.array( # double LEM_con[N_CONCEPTs][N_LEMMAs] = {
        # AN DU HO FU TAB CH DI EE PAA MEU TAF STO
        [[Y2, N, N, N, N, N,Y1, N, N, N, N, N],  # ANIMAL
         [ N,Y2, N, N, N, N, N,Y1, N, N, N, N],  # DUCK
         [ N, N,Y2, N, N, N, N, N,Y1, N, N, N],  # HORSE
         [ N, N, N,Y2, N, N, N, N, N,Y1, N, N],  # FURNITURE
         [ N, N, N, N,Y2, N, N, N, N, N,Y1, N],  # TABLE
         [ N, N, N, N, N,Y2, N, N, N, N, N,Y1]], # CHAIR
        dtype=np.float)

    return SEM_con, LEM_con


def make_layers():

    SEM_node_act = np.zeros((N_CONCEPTs,), dtype=np.float)  # double SEM_node_act[N_CONCEPTs]
    LEM_node_act = np.zeros((N_LEMMAs,), dtype=np.float)    # double LEM_node_act[N_LEMMAs];

    # input buffers
    #input_SEM = np.zeros((N_CONCEPTs,), dtype=np.float)  # double input_SEM[N_CONCEPTs];
    #input_LEM = np.zeros((N_LEMMAs,), dtype=np.float)    # double input_LEM[N_LEMMAs];
    
    return SEM_node_act, LEM_node_act #, input_SEM, input_LEM


In [3]:
#/*****************
# * MAIN ROUTINES *
# *****************/
def main(SEM_rate=SEM_rate, LEM_rate=LEM_rate):
    #print_heading()
    SIM_DATA = compute_SOA_functions(SEM_rate=SEM_rate, LEM_rate=LEM_rate)
    print_expectations_of_RT(SIM_DATA)
    return SIM_DATA


def compute_SOA_functions(SEM_rate=SEM_rate, LEM_rate=LEM_rate):
    
    SIM_DATA = np.zeros((N_DCONDs,), dtype=np.float)  # int SIM_DATA[N_DCONDs]

    SEM_con, LEM_con = make_weight()
    SEM_con *= SEM_rate
    LEM_con *= LEM_rate
    
    SEM_node_act, LEM_node_act = make_layers()
    
    mass = np.zeros((N_STEPs,), dtype=np.float)
    hazard = np.zeros((N_STEPs,), dtype=np.float)
    Survival = np.zeros((N_STEPs,), dtype=np.float)

    for dcond in range(N_DCONDs):  # dcond:= distractor type: REL, UNR, TRA, CTR
        SEM_node_act.fill(0.); LEM_node_act.fill(0.) # reset_network()
        #mass, hazard, Survival = reset_f_h_S_(mass, hazard, Survival)
        mass.fill(0.); hazard.fill(0.); Survival.fill(0.)
        
        for T in range(0, 1000, STEP_SIZE): 
            # stop for lemma retrieval (!!) latency > 1000 ms
            # truncation error will be marginal 
            # because for T>1000, f(s)<0.000001
            hazard = compute_hazard_rate(T, hazard, LEM_node_act)  
            SEM_node_act, LEM_node_act = update_network(T, dcond, \
                                                        SEM_con, LEM_con, \
                                                        SEM_node_act, LEM_node_act)
            print_networks_status(dcond, T, LEM_node_act)
            #print('harzad', hazard)
            
        Survival = compute_cumul_survival_function(hazard, Survival)
        mass = compute_mass_function(mass, hazard, Survival)
        SIM_DATA[dcond] = compute_expectation_of_RT_(mass)
        #print('S       :', Survival)
        #print('m       :', mass)
        #print('SIM_DATA:', SIM_DATA[dcond])
        
    SEM_con *= 1.0 / SEM_rate
    LEM_con *= 1.0 / LEM_rate
    return SIM_DATA


# Notice: hazard rates are computed before updating the network, that
# is, on the basis of the activation state of the network achieved at
# the previous time step (see Equation 2 in Roelofs, 1992, Cognition)
def set_spreading_rates(SEM_con, LEM_con):
    for i in range(N_CONCEPTs):
        for j in range(N_CONCEPTs):
            SEM_con[i][j] *= SEM_rate

    for i in range(N_CONCEPTs):
        for j in range(N_LEMMAs):
            LEM_con[i][j] *= LEM_rate
    return SEM_con, LEM_con


def reset_spreading_rates(SEM_con, LEM_con):
    # Needed, because otherwise in case of later tasks, if programmed, 
    # rates become a proportion of earlier ones; this routine 
    # keeps them fixed
    for i in range(N_CONCEPTs):
        for j in range(N_CONCEPTs):
            SEM_con[i][j ]*= 1.0/SEM_rate

    for i in range(N_CONCEPTs):
        for j in range(N_LEMMAs):
            LEM_con[i][j] *= 1.0/LEM_rate
    return SEM_con, LEM_con


def reset_network(SEM_node_act, LEM_node_act):
    for i in range(N_CONCEPTs):
        SEM_node_act[i] = 0.0

    for i in range(N_LEMMAs):
        LEM_node_act[i] = 0.0
    return SEM_node_act, LEM_node_act

In [4]:
#/*****************************
# * NETWORK UPDATING ROUTINES *
# *****************************/
def update_network(T, dcond, \
                   SEM_con, LEM_con, \
                   SEM_node_act, LEM_node_act):
    
    # input buffers
    input_SEM = np.zeros((N_CONCEPTs,), dtype=np.float)  # double input_SEM[N_CONCEPTs];
    input_LEM = np.zeros((N_LEMMAs,), dtype=np.float)    # double input_LEM[N_LEMMAs];

    input_SEM, input_LEM = get_external_input(T, dcond, input_SEM, input_LEM)
    input_SEM, input_LEM = get_internal_input(input_SEM, input_LEM, \
                                              SEM_con, LEM_con, \
                                              SEM_node_act, LEM_node_act)

    SEM_node_act, LEM_node_act = update_activation_of_nodes(input_SEM, SEM_node_act, \
                                                            input_LEM, LEM_node_act)
    
    return SEM_node_act, LEM_node_act
    

def set_input_to_zero():
    for i in range(N_CONCEPTs):
        input_SEM[i] = 0.0

    for i in range(N_LEMMAs):
        input_LEM[i] = 0.0


def get_external_input(T, dcond, input_SEM, input_LEM):
    # NOTE: verification times are not computed on-line, but simply set

    # target input
    if T >= 0:
        input_SEM[HORSE] += EXTIN  # entry picture
        # target concept gets signalling activation
        if T >= (0 + 1 * VERIFICATION_SPEED):  # 1 verification
            input_SEM[HORSE] += EXTIN

    # distractor word input
    if (T >= 0) and (T < DISTRACTOR_DURATION):
        if dcond == REL:
            input_LEM[EEND] += EXTIN
        if dcond == UNR: 
            input_LEM[STOEL] += EXTIN
        if dcond == TRA:
            input_LEM[PAARD] += EXTIN
        #if dcond == CTR:
        #    ;
    return input_SEM, input_LEM


# Equation 1 in paper Roelofs, 1992, Cognition
# $$ a(m, t+\Delta t) = a(m,t)(1-d) + \sum_{n\in N} w(n,m)a(n,t) $$
def get_internal_input(input_SEM, input_LEM, SEM_con, LEM_con, SEM_node_act, LEM_node_act):
    for i in range(N_CONCEPTs):
        for j in range(N_CONCEPTs):
            input_SEM[i] += SEM_node_act[j] * SEM_con[j][i]

    for i in range(N_CONCEPTs):
        for j in range(N_LEMMAs):
            input_SEM[i] += LEM_node_act[j] * LEM_con[i][j]

    for i in range(N_LEMMAs):
        for j in range(N_CONCEPTs):
            input_LEM[i] += SEM_node_act[j] * LEM_con[j][i]
            
    return input_SEM, input_LEM


#def update_activation_of_nodes(T, dcond, input_SEM, input_LEM, SEM_node_act, LEM_node_act):
def update_activation_of_nodes(input_SEM, SEM_node_act, input_LEM, LEM_node_act):
    # Equation 1 in paper, Roelofs, 1992

    SEM_node_act *= (1.0 - DECAY_rate)
    SEM_node_act += input_SEM
    #for i in range(N_CONCEPTs):
    #    SEM_node_act[i] = (SEM_node_act[i] * (1.0 - DECAY_rate)) + input_SEM[i]

    LEM_node_act *= (1.0 - DECAY_rate)
    LEM_node_act += input_LEM
    #for i in range(N_LEMMAs):
    #    LEM_node_act[i] = (LEM_node_act[i] * (1.0 - DECAY_rate)) + input_LEM[i]

    #print('+input_LEM   :', input_LEM)
    #print('+LEM_node_act:', LEM_node_act)
    #sys.exit()
    return SEM_node_act, LEM_node_act


def print_networks_status(dcond, T, LEM_node_act):
    # Here the node activations in the related, unrelated, and translation conditions are printed.
    # For convenience, the printing is requested here rather than in a separate routine.
    # In Fig 6 of the article, 200 ms has been added to cover the lead-in processes, i.e.,
    # picture and visual word processing, thus time in the figure starts at 200 ms.
    # Printed are the lemma activations of the Dutch distractors (left panel of Fig 6),
    # the English competitors (middle panel), and English target (right panel)
    if (T < 275) and (dcond != CTR):
        if T == 0:
            print("Cond   Time   Node activations (Fig 6 in article)")

        if dcond == REL:
            print("REL    {0:3d}    EEND:{1:5.2f}   DUCK:{2:5.2f}  HORSE:{3:5.2f}".format(
                T, LEM_node_act[EEND], LEM_node_act[DUCK], LEM_node_act[HORSE]))
            figREL[T] = {'eend': LEM_node_act[EEND],
                         'duck': LEM_node_act[DUCK],
                         'horse': LEM_node_act[HORSE]}
        if dcond == UNR:
            print("UNR    {0:3d}   STOEL:{1:5.2f}  CHAIR:{2:5.2f}  HORSE:{3:5.2f}".format(
                T, LEM_node_act[STOEL], LEM_node_act[CHAIR], LEM_node_act[HORSE]))
            figUNR[T] = {'stoel': LEM_node_act[STOEL],
                         'chair': LEM_node_act[CHAIR],
                         'horse': LEM_node_act[HORSE]}
        if dcond == TRA:
            print("TRA    {0:3d}   PAARD:{1:5.2f}  HORSE:{2:5.2f}".format(
                T, LEM_node_act[PAARD], LEM_node_act[HORSE]))
            figTRA[T] = {'paard': LEM_node_act[PAARD],
                         'horse': LEM_node_act[HORSE]}


In [5]:
#/*********************************************************
# *  HAZARD-, CUMUL_SURVIVAL-, MASS-FUNCTION ROUTINES     *
# *********************************************************/
#    h[1] denotes hazard rate at first time step,
#    h[2] denotes hazard rate at second time step, etc.,
#    where h[s] is a function (expressed by the Luce ratio) of the
#    activation levels at moment T equal to (s-1) * STEP_SIZE (where s=1,2,...)
#
#    f[1] denotes prob of selection at first time step, etc;
#    that is, f[s] is the probability that the retrieval latency
#    equals s x STEP_SIZE;
#    e.g., f[2] is the probability that the retrieval latency equals
#    2 x STEP_SIZE, i.e., 2 x 25 ms = 50 ms.
def compute_hazard_rate(T, hazard, LEM_node_act):
    # NOTE: verification times are not
    # computed on-line, but simply set.
    if T >= 0:
        i = int(T/STEP_SIZE + 1)
        #i = int(T/STEP_SIZE) + 1
        # Central claim: competition for selection is within language, 
        # i.e., the naming language L2 English
        # 2 verifications
        if (T >= (0 + 2 * VERIFICATION_SPEED)) \
        and ((LEM_node_act[HORSE] - LEM_node_act[DUCK]) > CRIT_DIFF) \
        and ((LEM_node_act[HORSE] - LEM_node_act[TABLE]) > CRIT_DIFF) \
        and ((LEM_node_act[HORSE] - LEM_node_act[CHAIR]) > CRIT_DIFF):
            # in denominator the lemma nodes of words in the target language
            denominator = LEM_node_act[HORSE] + LEM_node_act[DUCK] + LEM_node_act[TABLE] + LEM_node_act[CHAIR]
            #hazard[i] = LEM_node_act[HORSE] / LEM_node_act.sum()
            hazard[i] = LEM_node_act[HORSE] / denominator
            #print('T:', T, 'i:', i, 'harzard[i]:', hazard[i])
            #print('LEM_node_act[HORSE]:', LEM_node_act[HORSE])
            #print('denominator:', denominator)
            #sys.exit()
        else:
            hazard[i] = 0.0
    return hazard
            

def compute_cumul_survival_function(hazard, Survival):
    """NOTE: cum_survival or S[s] is upto and including s"""
    for s in range(N_STEPs):
        aux = 1.0
        for j in range(s+1):
            aux *= (1.0 - hazard[j])
        Survival[s] = aux
    return Survival


def compute_mass_function(mass, hazard, Survival):
    """
    NOTE: Prob(not selected before step s) equals S[s-1], that is
    surviving upto and including the previous time step
    """
    # NOTE: f[0] will always be 0, so does not have to be computed
    for s in range(1, N_STEPs):
        mass[s] = hazard[s] * Survival[s-1]
    return mass


def compute_expectation_of_RT_(mass):
    mean = 0.
    for s in range(N_STEPs):
        mean += mass[s] * s * STEP_SIZE

    return mean


def reset_f_h_S_(mass, hazard, Survival):
    mass.fill(0.); hazard.fill(0.); Survival.fill(0.)
    #for s in range(N_STEPs):
    #    f[s] = 0.0
    #    h[s] = 0.0
    #    S[s] = 0.0
    return mass, hazard, Survival

In [6]:
#/*****************
# *  I/0 ROUTINES *
# *****************/
def print_heading():
    print("Simulation of Bilingual Lemma Retrieval in Speaking (c) Ardi Roelofs");
    #print("working...")


def print_expectations_of_RT(SIM_DATA):
    
    print("\nSimulated lemma retrieval time and effects (Fig 5 in article)");
    print("REL   UNR   TRA   CTR   Sem   Tra effects")
    print("{0:.0f}   {1:.0f}   {2:.0f}   {3:.0f}   {4:.0f}   {5:.0f}".format(
        SIM_DATA[REL], SIM_DATA[UNR], SIM_DATA[TRA], SIM_DATA[CTR], 
        SIM_DATA[REL] - SIM_DATA[UNR],
        SIM_DATA[TRA] - SIM_DATA[UNR]))

    print("\nParameter values:");
    print("verfication speed  : {:6d} [ms]".format(VERIFICATION_SPEED))
    print("sem_rate           : {:.4f} [prop/ms]".format(SEM_rate/STEP_SIZE))
    print("lem_rate           : {:.4f} [prop/ms]".format(LEM_rate/STEP_SIZE))
    print("external input     : {:.4f} [act_units/ms]".format(EXTIN/STEP_SIZE))
    print("decay per step     : {:.4f} [prop/ms]".format(DECAY_rate/STEP_SIZE))
    print("distractor duration: {:6d} [ms]".format(DISTRACTOR_DURATION))
    print("critical difference: {:.4f} [act_units]".format(CRIT_DIFF))
    #print("\npress any key to continue ");
    #getchar();

In [7]:
SEM_rate = 0.0101 * STEP_SIZE    # prop per step_size ms
LEM_rate = 0.0074 * STEP_SIZE    # prop per step_size ms
figREL, figUNR, figTRA = {}, {}, {}
Results = main()
#main(SEM_rate=SEM_rate, LEM_rate=LEM_rate)
main()

Cond   Time   Node activations (Fig 6 in article)
REL      0    EEND: 4.91   DUCK: 0.00  HORSE: 0.00
REL     25    EEND: 6.88   DUCK: 0.00  HORSE: 0.45
REL     50    EEND: 8.06   DUCK: 0.20  HORSE: 1.27
REL     75    EEND: 9.14   DUCK: 0.58  HORSE: 1.95
REL    100    EEND: 5.25   DUCK: 1.03  HORSE: 2.54
REL    125    EEND: 4.28   DUCK: 1.50  HORSE: 3.08
REL    150    EEND: 4.29   DUCK: 1.89  HORSE: 3.58
REL    175    EEND: 4.71   DUCK: 2.25  HORSE: 4.04
REL    200    EEND: 5.31   DUCK: 2.61  HORSE: 4.47
REL    225    EEND: 5.99   DUCK: 2.98  HORSE: 4.87
REL    250    EEND: 6.69   DUCK: 3.34  HORSE: 5.25
Cond   Time   Node activations (Fig 6 in article)
UNR      0   STOEL: 4.91  CHAIR: 0.00  HORSE: 0.00
UNR     25   STOEL: 6.88  CHAIR: 0.00  HORSE: 0.45
UNR     50   STOEL: 7.83  CHAIR: 0.08  HORSE: 1.27
UNR     75   STOEL: 8.35  CHAIR: 0.18  HORSE: 1.93
UNR    100   STOEL: 3.75  CHAIR: 0.28  HORSE: 2.48
UNR    125   STOEL: 2.01  CHAIR: 0.37  HORSE: 2.97
UNR    150   STOEL: 1.23  CHAIR: 

array([213.57, 162.00, 130.43, 158.17])

# 出力サンプル

```text
Cond   Time   Node activations (Fig 6 in article)
REL      0    EEND:4.91  DUCK:0.00 HORSE:0.00 
REL     25    EEND:6.88  DUCK:0.00 HORSE:0.45 
REL     50    EEND:8.06  DUCK:0.20 HORSE:1.27 
REL     75    EEND:9.14  DUCK:0.58 HORSE:1.95 
REL    100    EEND:5.25  DUCK:1.03 HORSE:2.54 
REL    125    EEND:4.28  DUCK:1.50 HORSE:3.08 
REL    150    EEND:4.29  DUCK:1.89 HORSE:3.58 
REL    175    EEND:4.71  DUCK:2.25 HORSE:4.04 
REL    200    EEND:5.31  DUCK:2.61 HORSE:4.47 
REL    225    EEND:5.99  DUCK:2.98 HORSE:4.87 
REL    250    EEND:6.69  DUCK:3.34 HORSE:5.25 
Cond   Time   Node activations (Fig 6 in article)
UNR      0    STOEL:4.91 CHAIR:0.00 HORSE:0.00 
UNR     25    STOEL:6.88 CHAIR:0.00 HORSE:0.45 
UNR     50    STOEL:7.83 CHAIR:0.08 HORSE:1.27 
UNR     75    STOEL:8.35 CHAIR:0.18 HORSE:1.93 
UNR    100    STOEL:3.75 CHAIR:0.28 HORSE:2.48 
UNR    125    STOEL:2.01 CHAIR:0.37 HORSE:2.97 
UNR    150    STOEL:1.23 CHAIR:0.36 HORSE:3.41 
UNR    175    STOEL:0.86 CHAIR:0.33 HORSE:3.83 
UNR    200    STOEL:0.68 CHAIR:0.30 HORSE:4.24 
UNR    225    STOEL:0.59 CHAIR:0.28 HORSE:4.63 
UNR    250    STOEL:0.54 CHAIR:0.26 HORSE:5.01 
Cond   Time   Node activations (Fig 6 in article)
TRA      0    PAARD:4.91 HORSE:0.00 
TRA     25    PAARD:7.79 HORSE:0.45 
TRA     50    PAARD:10.38 HORSE:1.36 
TRA     75    PAARD:12.21 HORSE:2.12 
TRA    100    PAARD:8.71 HORSE:2.76 
TRA    125    PAARD:7.94 HORSE:3.33 
TRA    150    PAARD:8.05 HORSE:3.77 
TRA    175    PAARD:8.53 HORSE:4.16 
TRA    200    PAARD:9.16 HORSE:4.54 
TRA    225    PAARD:9.85 HORSE:4.91 
TRA    250    PAARD:10.56 HORSE:5.27 

Simulated lemma retrieval time and effects (Fig 5 in article) 
REL     UNR     TRA     CTR        Sem     Tra effects
213     162     130     158        51     -32

Parameter values:
ver speed :     25 [ms]
sem_rate  : 0.0101 [prop/ms]
lem_rate  : 0.0074 [prop/ms]
exin      : 0.1965 [act_units/ms]
d         : 0.0240 [prop/ms]
du        :    100 [ms]
cd        : 1.6000 [act_units]
```

In [8]:
# The graph that was depicted on the original paper, Reolofs (2016)
fig6_1 = np.array( #time  eend Duck Horse (Related)
    [[  0, 4.91, 0.00, 0.00],
     [ 25, 6.88, 0.00, 0.45],
     [ 50, 8.06, 0.20, 1.27], 
     [ 75, 9.14, 0.58, 1.95], 
     [100, 5.25, 1.03, 2.54],  
     [125, 4.28, 1.50, 3.08],
     [150, 4.29, 1.89, 3.58], 
     [175, 4.71, 2.25, 4.04],  
     [200, 5.31, 2.61, 4.47],
     [225, 5.99, 2.98, 4.87],
     [250, 6.69, 3.34, 5.25]])

fig6_2 = np.array( #Time   Stoel Chair Horse Unrelated
    [[  0, 4.91, 0.00, 0.00],
     [ 25, 6.88, 0.00, 0.45],
     [ 50, 7.83, 0.08, 1.27],
     [ 75, 8.35, 0.18, 1.93],
     [100, 3.75, 0.28, 2.48],
     [125, 2.01, 0.37, 2.97],
     [150, 1.23, 0.36, 3.41],
     [175, 0.86, 0.33, 3.83],
     [200, 0.68, 0.30, 4.24],
     [225, 0.59, 0.28, 4.63],
     [250, 0.54, 0.26, 5.01]])
fig6_3 = np.array( #Time  Paard  Horse translated
    [[  0, 4.91, 0.00],
     [ 25, 7.79, 0.45],
     [ 50,10.38, 1.36],
     [ 75,12.21, 2.12],
     [100, 8.71, 2.76],
     [125, 7.94, 3.33],
     [150, 8.05, 3.77],
     [175, 8.53, 4.16],
     [200, 9.16, 4.54],
     [225, 9.85, 4.91],
     [250,10.56, 5.27]])

#print(fig6_1)
X = fig6_1.T[0]

def draw_fig6_left():
    plt.plot(X, fig6_1.T[1], 'ro', label='eend:semantic')
    plt.plot(X, fig6_1.T[1], 'r-')
    plt.plot(X, fig6_2.T[1], 'bo', label='stoel:unrelated')
    plt.plot(X, fig6_2.T[1], 'b-')
    plt.plot(X, fig6_3.T[1], 'go', label='paard:traslation')
    plt.plot(X, fig6_3.T[1], 'g-')
    plt.legend()
    plt.ylim([0,14])
    plt.title('Dutch dsitractors')
    plt.show()

def draw_fig6_center():
    plt.plot(X, fig6_1.T[2], 'ro', label='duck:semantic')
    plt.plot(X, fig6_1.T[2], 'r-')
    plt.plot(X, fig6_2.T[2], 'bo', label='chair:unrelated')
    plt.plot(X, fig6_2.T[2], 'b-')
    plt.legend()
    plt.ylim([0,6])
    plt.title('English competitions')
    plt.show()

def draw_fig6_right():
    plt.plot(X, fig6_1.T[3], 'ro', label='semantic')
    plt.plot(X, fig6_1.T[3], 'r-')
    plt.plot(X, fig6_2.T[3], 'bo', label='unrelated')
    plt.plot(X, fig6_2.T[3], 'b-')
    plt.plot(X, fig6_3.T[2], 'go', label='translated')
    plt.plot(X, fig6_3.T[2], 'g-')
    plt.legend()
    plt.ylim([0,6])
    plt.title('English target (horse)')
    plt.show()

def draw_fig5():
    data = np.array(# Simulated lemma retrieval time and effects (Fig 5 in article) 
    # Related Unrelated Translation CTR Semantc Traslation, effects
    [213, 162, 130, 158,  51, -32])
    plt.bar([0,1], data[4:])
    plt.ylim(-40,60)
    plt.xticks([0, 1], ['semantic', 'translation'])
    plt.show()

#[draw_fig6_left(), draw_fig6_center(), draw_fig6_right(), draw_fig5()]
#figREL, figUNR, figTRA = {}, {}, {}

In [ ]:
# The results calculated in this code
X = [T for T in range(0, 251, STEP_SIZE)]
nfigREL = np.zeros((len(X),3), dtype=np.float)
nfigUNR = np.zeros((len(X),3), dtype=np.float)
nfigTRA = np.zeros((len(X),2), dtype=np.float)

for fig, data in zip([figREL, figUNR, figTRA],[nfigREL, nfigUNR, nfigTRA]):
    for i, T in enumerate(fig.keys()):
        for j, word in enumerate(fig[T].keys()):
            data[i][j] = fig[T][word]


def _fig6_left():
    plt.plot(X, nfigREL.T[0], 'ro', label='eend:semantic')
    plt.plot(X, nfigREL.T[0], 'r-')
    plt.plot(X, nfigUNR.T[0], 'bo', label='stoel:unrelated')
    plt.plot(X, nfigUNR.T[0], 'b-')
    plt.plot(X, nfigTRA.T[0], 'go', label='paard:traslation')
    plt.plot(X, nfigTRA.T[0], 'g-')
    plt.legend()
    plt.ylim([0,14])
    plt.title('Dutch dsitractors')
    plt.ylabel('Activation (activation unit)')
    plt.xlabel('Time (ms)')
    plt.title('Fig.6. Activation curves for the lemmas of Dutch word distractors, English competitors, and the English target in WEAVER++ simulation.')
    plt.show()

def _fig6_center():
    plt.plot(X, nfigREL.T[1], 'ro', label='duck:semantic')
    plt.plot(X, nfigREL.T[1], 'r-')
    plt.plot(X, nfigUNR.T[1], 'bo', label='chair:unrelated')
    plt.plot(X, nfigUNR.T[1], 'b-')
    plt.legend()
    plt.ylim([0,6])
    plt.title('English competitions')
    plt.ylabel('Activation (activation unit)')
    plt.xlabel('Time (ms)')
    plt.title('Fig.6. Activation curves for the lemmas of Dutch word distractors, English competitors, and the English target in WEAVER++ simulation.')
    plt.show()

def _fig6_right():
    plt.plot(X, nfigREL.T[2], 'ro', label='semantic')
    plt.plot(X, nfigREL.T[2], 'r-')
    plt.plot(X, nfigUNR.T[2], 'bo', label='unrelated')
    plt.plot(X, nfigUNR.T[2], 'b-')
    plt.plot(X, nfigTRA.T[1], 'go', label='translated')
    plt.plot(X, nfigTRA.T[1], 'g-')
    plt.legend()
    plt.ylim([0,6])
    plt.title('English target (horse)')
    plt.xlabel('Time (ms)')
    plt.ylabel('Activation (activation unit)')
    plt.title('Fig.6. Activation curves for the lemmas of Dutch word distractors, English competitors, and the English target in WEAVER++ simulation.')
    plt.show()

def _fig5():
    Sem = Results[0] - Results[1]
    Tra = Results[2] - Results[1]
    plt.bar([0, 1], [Sem, Tra])
    plt.ylim(-40,60)
    plt.xticks([0, 1], ['semantic', 'translation'])
    plt.ylabel('Latency difference (ms)')
    plt.xlabel('Effect type')
    plt.title('Fig. 5. Semantic and translation effecs on the latency of lemma retrieval in WEAVER++.')
    plt.show()

    
# 以下の行頭の記号 # を削除することでグラフが描画されます
# `draw_fig` で 始まる関数は 元論文の図を描画し
# `_fig` で始まる関数は 本シミュレーションの結果を描画します

# [_fig6_left(), _fig6_center(), _fig6_right()]
#[draw_fig6_left(), draw_fig6_center(), draw_fig6_right(), draw_fig5()]
draw_fig6_left(); 
_fig6_left()

draw_fig6_center(); 
_fig6_center()

draw_fig6_right(); 
_fig6_right()

_fig5()


# オリジナルソースコード

```C++
/***************************************************
 *                                                 *
 *  ERP_bilingual_lemret.c                         *
 *                                                 *
 *  SIMULATION OF LEMMA RETRIEVAL IN SPEAKING      *
 *                                                 *
 *  BILINGUAL NAMING                               *
 *                                                 *
 *  Ardi Roelofs                                   *
 *                                                 *
 *  Published in:                                  *
 *  Roelofs, A., Piai, V., Garrido Rodriguez, G.,  *
 *  & Chwilla, D. J. (2016). Electrophysiology of  *
 *	cross-language interference and facilitation   *
 *	in picture naming. Cortex, 76, 1-16.           *
 *                                                 *
 *  (C) Copyright DCC                              *
 *                                                 *
 ***************************************************/

 /*
  A brief note on C and programming style:

  This program is written in the C programming language, which is
  described by Kernighan and Ritchie (1988) and many others. C was
  chosen because it is among the most widely and frequently used
  programming languages, and C programs can be compiled (using
  freeware C and C++ compilers) for all main computer platforms
  and operating systems.

  Following Kernighan and Plauger's (1978) maxim "write clearly
  -- don't be too clever", I avoided the use of pointers
  and other initially somewhat obscure constructs to help readers
  unfamiliar with C. Given that the program is rather small
  (everything is in one file), I chose for external variables
  that are globally accessible to all functions rather than
  using function arguments and return values for communicating
  data between functions. External variables are more convenient
  and efficient here than long argument lists.

  Kernighan, B.W., & Plauger, P.J. (1978). The Elements of
  Programming Style (Second Edition). New York: McGraw Hill.

  Kernighan, B.W., & Ritchie, D.M. (1988). The C Programming Language
  (Second Edition). Englewood Cliffs, NJ: Prentice Hall.
 */


#include <stdio.h>
#include <stdlib.h>
#include <float.h>


#define STEP_SIZE 25   /* duration time step in ms */
#define N_STEPs (1000+STEP_SIZE)/STEP_SIZE /* trunc after 41 steps */
#define N_CONCEPTs 6
#define N_LEMMAs 12
#define N_DCONDs 4
#define Y 1.0    /* connection present */
#define Y1 1.0   /* connection present, L1 */
#define Y2 0.5   /* connection present, weaker L2 */
#define N 0.0    /* connection absent */
#define REL 0    /* semantically related condition */
#define UNR 1    /* unrelated condition */
#define TRA 2    /* translation condition */
#define CTR 3    /* control condition */


 /* labeling network nodes */
 /* L2 English, the target language for picture naming */
#define ANIMAL 0
#define DUCK 1
#define HORSE 2
#define FURNITURE 3
#define TABLE 4
#define CHAIR 5
 /* L1 Dutch, the language of the distractor words */ 
 /* Dutch words are translation equivalents of the English words */
#define DIER 6
#define EEND 7
#define PAARD 8
#define MEUBEL 9
#define TAFEL 10
#define STOEL 11

 /* connections conceptual network, shared between languages */
 double SEM_con[N_CONCEPTs][N_CONCEPTs] =  {
	           /*     ANIMAL DUCK HORSE FURN TABLE CHAIR */
 /* ANIMAL     */   {   N,    Y,   Y,   N,    N,   N },
 /* DUCK       */   {   Y,    N,   Y,   N,    N,   N },
 /* HORSE      */   {   Y,    Y,   N,   N,    N,   N },
 /* FURNITURE  */   {   N,    N,   N,   N,    Y,   Y },
 /* TABLE      */   {   N,    N,   N,   Y,    N,   Y },
 /* CHAIR      */   {   N,    N,   N,   Y,    Y,   N }
 };


 /* connections between concept and lemma nodes */
 double LEM_con[N_CONCEPTs][N_LEMMAs] = {
                 /* AN  DU  HO  FU  TAB CH  DI  EE  PAA  MEU TAF STO */
/* ANIMAL    */   { Y2,  N,  N,  N,  N,  N, Y1,  N,  N,  N,  N,  N },
/* DUCK      */   { N,  Y2,  N,  N,  N,  N,  N,  Y1, N,  N,  N,  N },
/* HORSE     */   { N,   N, Y2,  N,  N,  N,  N,  N,  Y1, N,  N,  N },
/* FURNITURE */   { N,   N,  N,  Y2, N,  N,  N,  N,  N,  Y1, N,  N },
/* TABLE     */   { N,   N,  N,  N,  Y2, N,  N,  N,  N,  N,  Y1, N }, 
/* CHAIR     */   { N,   N,  N,  N,  N,  Y2, N,  N,  N,  N,  N,  Y1 }
 };

 double SEM_node_act[N_CONCEPTs], LEM_node_act[N_LEMMAs];

 /* input buffers */
 double input_SEM[N_CONCEPTs];
 double input_LEM[N_LEMMAs];

 int T;     /* time in ms */

 int dcond; /* distractor type: REL, UNR, TRA, CTR */


 /* parameter set */
 int    VERIFICATION_SPEED = 25;         /* ms per link */
 double SEM_rate = 0.0101 * STEP_SIZE;   /* prop per step_size ms */
 double LEM_rate = 0.0074 * STEP_SIZE;   /* prop per step_size ms */
 double DECAY_rate = 0.0240 * STEP_SIZE; /* prop per step_size ms */
 double EXTIN = 0.1965 * STEP_SIZE;      /* act_units per step_size ms */
 int    DISTRACTOR_DURATION = 100;       /* ms */
 double CRIT_DIFF = 1.6;                 /* act_units */

 int SIM_DATA[N_DCONDs];
 
 double h[N_STEPs];
 double S[N_STEPs];
 double f[N_STEPs];


 void reset_network();
 void set_spreading_rates();
 void reset_spreading_rates();
 void update_network();
 void set_input_to_zero();
 void get_external_input();
 void get_internal_input();
 void update_activation_of_nodes();

 void reset_system();
 void reset_f_h_S_();
 void compute_SOA_functions();
 void compute_hazard_rate();
 void compute_cumul_survival_function();
 void compute_mass_function();
 double compute_expectation_of_RT_();

 void print_heading();
 void print_expectations_of_RT();




/*****************
 * MAIN ROUTINES *
 *****************/

 int main(void)
 {

	   reset_system();

	   set_spreading_rates();

	   print_heading();

       compute_SOA_functions();

	   print_expectations_of_RT();

       reset_spreading_rates();
 }


 void compute_SOA_functions()
 {

   for(dcond=0;dcond<N_DCONDs;dcond++) {
	   reset_network();
	   reset_f_h_S_();
	   for(T=0;T<1000;T+=STEP_SIZE) { /* stop for lemma retrieval (!!)     */
	     compute_hazard_rate();       /* latency > 1000 ms                 */
	     update_network();            /* truncation error will be marginal */
	     }                            /* because for T>1000, f(s)<0.000001 */
	   compute_cumul_survival_function();
	   compute_mass_function();
	   SIM_DATA[dcond]= (int) compute_expectation_of_RT_();
   }
 }
 /* Notice: hazard rates are computed before updating the network, that
    is, on the basis of the activation state of the network achieved at
    the previous time step (see Equation 2 in Roelofs, 1992, Cognition) */


 void reset_system()
 {
     int i;

     for(i=0;i<N_DCONDs;i++)
	    SIM_DATA[i]=0;
 }


 void set_spreading_rates()
 {
   int i,j;

   for(i=0;i<N_CONCEPTs;i++)
     for(j=0;j<N_CONCEPTs;j++) {
	SEM_con[i][j]*=SEM_rate;
	}

 
   for(i=0;i<N_CONCEPTs;i++)
     for(j=0;j<N_LEMMAs;j++) {
	LEM_con[i][j]*=LEM_rate;
	}
 
 }



 void reset_spreading_rates()
 {
 /* Needed, because otherwise in case of later tasks, if programmed, 
    rates become a proportion of earlier ones; this routine 
	keeps them fixed */

   int i,j;

   for(i=0;i<N_CONCEPTs;i++)
     for(j=0;j<N_CONCEPTs;j++) {
	SEM_con[i][j]*=(1.0/SEM_rate);
	}

   for(i=0;i<N_CONCEPTs;i++)
     for(j=0;j<N_LEMMAs;j++) {
	LEM_con[i][j]*=(1.0/LEM_rate);
	}
 
 }

 void reset_network()
 {
   int i;

   for(i=0;i<N_CONCEPTs;i++) {
     SEM_node_act[i]=0.0;
     }

    for(i=0;i<N_LEMMAs;i++) {
     LEM_node_act[i]=0.0;
     }
 
 
 }




/*****************************
 * NETWORK UPDATING ROUTINES *
 *****************************/

 void update_network()
 {
   set_input_to_zero();
   get_external_input();
   get_internal_input();
   update_activation_of_nodes();
 }


 void set_input_to_zero()
 {
   int i;

   for(i=0;i<N_CONCEPTs;i++) {
       input_SEM[i]=0.0;
       }

   for(i=0;i<N_LEMMAs;i++) {
       input_LEM[i]=0.0;
       }
 
 }


 /* Equation 1 in paper Roelofs, 1992, Cognition*/

 void get_external_input()
 {

 /* NOTE: verification times are not computed on-line, but simply set  */


    /* target input */
    if(T>=0) {
		/* entry picture */
	   input_SEM[HORSE]+=EXTIN;


	/* target concept gets signalling activation */
      if(T >= (0 + 1 * VERIFICATION_SPEED) ) /* 1 verification */
	   input_SEM[HORSE]+=EXTIN;

    }




    /* distractor word input */
    if(T >= 0 && T < DISTRACTOR_DURATION) {

	   if(dcond==REL) 
	     input_LEM[EEND]+=EXTIN;

	   if(dcond==UNR) 
	     input_LEM[STOEL]+=EXTIN;

	   if(dcond==TRA) 
	     input_LEM[PAARD]+=EXTIN;

	   if(dcond==CTR)
                                           ;
        }
	
	}
 



 void get_internal_input()
 {
   int i,j;

   for(i=0;i<N_CONCEPTs;i++)
     for(j=0;j<N_CONCEPTs;j++) {
       input_SEM[i]+=( SEM_node_act[j] * SEM_con[j][i]);
       }

 
   for(i=0;i<N_CONCEPTs;i++)
     for(j=0;j<N_LEMMAs;j++) {
       input_SEM[i]+= (LEM_node_act[j] * LEM_con[i][j] ); 

       } 
 

   for(i=0;i<N_LEMMAs;i++)
     for(j=0;j<N_CONCEPTs;j++) {
       input_LEM[i]+=( SEM_node_act[j] * LEM_con[j][i] );
	 }

 
 }


 void update_activation_of_nodes()
 {
   int i;

   /* Equation 1 in paper, Roelofs, 1992 */
   for(i=0;i<N_CONCEPTs;i++) {
     SEM_node_act[i]=((SEM_node_act[i] * (1.0 - DECAY_rate)) + input_SEM[i]);
     }

 
   for(i=0;i<N_LEMMAs;i++) {
     LEM_node_act[i]=((LEM_node_act[i] * (1.0 - DECAY_rate)) + input_LEM[i]);
     }

   /* Here the node activations in the related, unrelated, and translation conditions are printed.
      For convenience, the printing is requested here rather than in a separate routine.
      In Fig 6 of the article, 200 ms has been added to cover the lead-in processes, i.e.,
      picture and visual word processing, thus time in the figure starts at 200 ms.
	  Printed are the lemma activations of the Dutch distractors (left panel of Fig 6),
	  the English competitors (middle panel), and English target (right panel) */
   if(T < 275 && dcond!=CTR) {

	if (T == 0) 
	   printf("Cond   Time   Node activations (Fig 6 in article)\n");

   if(dcond==REL) 
	   printf("REL    %3d    EEND:%3.2f  DUCK:%3.2f HORSE:%3.2f \n", T, LEM_node_act[EEND], 
		                                                                LEM_node_act[DUCK], 
		                                                                LEM_node_act[HORSE]);
   if(dcond==UNR)
	   printf("UNR    %3d    STOEL:%3.2f CHAIR:%3.2f HORSE:%3.2f \n", T, LEM_node_act[STOEL], 
		                                                                 LEM_node_act[CHAIR], 
		                                                                 LEM_node_act[HORSE]);
   if(dcond==TRA)
	   printf("TRA    %3d    PAARD:%3.2f HORSE:%3.2f \n", T, LEM_node_act[PAARD], LEM_node_act[HORSE]);
   }

 
 
 }



/*********************************************************
 *  HAZARD-, CUMUL_SURVIVAL-, MASS-FUNCTION ROUTINES     *
 *********************************************************/


/*
    h[1] denotes hazard rate at first time step,
    h[2] denotes hazard rate at second time step, etc.,
    where h[s] is a function (expressed by the Luce ratio) of the
    activation levels at moment T equal to (s-1) * STEP_SIZE (where s=1,2,...)

    f[1] denotes prob of selection at first time step, etc;
    that is, f[s] is the probability that the retrieval latency
    equals s x STEP_SIZE;
    e.g., f[2] is the probability that the retrieval latency equals
    2 x STEP_SIZE, i.e., 2 x 25 ms = 50 ms.
*/



 void compute_hazard_rate()
 {


 /* NOTE: verification times are not
	  computed on-line, but simply set.
 */

   if(T>=0) {

	   /* Central claim: competition for selection is within language, 
	      i.e., the naming language L2 English */

	 if( T >= (0+ 2 * VERIFICATION_SPEED)   /* 2 verifications */
	    &&
	   ( (LEM_node_act[HORSE] - LEM_node_act[DUCK]) > CRIT_DIFF )
	    &&
	   ( (LEM_node_act[HORSE] - LEM_node_act[TABLE]) > CRIT_DIFF )
	    &&
	   ( (LEM_node_act[HORSE] - LEM_node_act[CHAIR]) > CRIT_DIFF ) )

		/* in denominator the lemma nodes of words in the target language */
	    h[(T/STEP_SIZE)+1] = LEM_node_act[HORSE]  /
			       ( LEM_node_act[HORSE]
			       + LEM_node_act[DUCK]
			       + LEM_node_act[TABLE]
			       + LEM_node_act[CHAIR]);
	 else
	    h[(T/STEP_SIZE)+1] = 0.0;
      }


 }


 void compute_cumul_survival_function()
 {
     int j,s;
     double aux;

     /* NOTE: cum_survival or S[s] is upto and including s */

     for(s=0;s<N_STEPs;s++) {
       for(j=0, aux=1.0;j<=s;j++)
	  aux*=(1.0-h[j]);
       S[s]=aux;
       }
 }

 void compute_mass_function()
 {
     int s;

     /* NOTE: Prob(not selected before step s) equals S[s-1], that is
	surviving upto and including the previous time step */

     for(s=1;s<N_STEPs;s++)
       f[s]=h[s] * S[s-1];

     /* NOTE: f[0] will always be 0, so does not have to be computed */
 }

 double compute_expectation_of_RT_()
 {
     int s;
     double mean=0.0;

     for(s=0;s<N_STEPs;s++)
	  mean+=f[s] * s * STEP_SIZE;

     return mean;
 }

 void reset_f_h_S_()
 {
    int s;

    for(s=0;s<N_STEPs;s++) {
      f[s]=0.0;
      h[s]=0.0;
      S[s]=0.0;
      }
 }


/*****************
 *  I/0 ROUTINES *
 *****************/

 void print_heading()
 {

   printf("Simulation of Bilingual Lemma Retrieval in Speaking (c) Ardi Roelofs\n");
   printf("working...\n");

 }


 void print_expectations_of_RT()
 {
   
     printf("\nSimulated lemma retrieval time and effects (Fig 5 in article) \n");
     printf("REL     UNR     TRA     CTR        Sem     Tra effects\n");
	 printf("%3d     %3d     %3d     %3d       %3d     %3d\n", SIM_DATA[REL], SIM_DATA[UNR],
		 SIM_DATA[TRA], SIM_DATA[CTR], SIM_DATA[REL] - SIM_DATA[UNR], 
		 SIM_DATA[TRA] - SIM_DATA[UNR]
		 );
	 printf("\n");
 

     printf("\nParameter values:\n");
     printf("ver speed : %6d [ms]\n", VERIFICATION_SPEED);
     printf("sem_rate  : %.4f [prop/ms]\n",SEM_rate/STEP_SIZE);
     printf("lem_rate  : %.4f [prop/ms]\n",LEM_rate/STEP_SIZE);
     printf("exin      : %.4f [act_units/ms]\n",EXTIN/STEP_SIZE);
     printf("d         : %.4f [prop/ms]\n",DECAY_rate/STEP_SIZE);
     printf("du        : %6d [ms]\n",DISTRACTOR_DURATION);
     printf("cd        : %.4f [act_units]\n",CRIT_DIFF);

     printf("\npress any key to continue ");

     getchar();
	 
 }
```